In [ ]:
!cd

## 결측치 제거

In [1]:
# 새출발
import pandas as pd
import pymysql.cursors
import numpy as np

In [ ]:
sql="select customers.customerName, payments.checkNumber from customers left join payments on customers.customerNumber=payments.customerNumber where payments.paymentDate >= '2004-10-06';"


In [2]:
# MySQL DB에서 데이터 받아와서 DataFrame에 저장

conn = pymysql.connect(host='localhost', user='root', 
                       password='054867', db='tip', charset='utf8',
                       autocommit=True, cursorclass=pymysql.cursors.DictCursor)

In [3]:
try:

   with conn.cursor() as curs:
      sql="SELECT * FROM tips;"
      curs.execute(sql)
      rs = curs.fetchall()

      # DB에서 받아온 값을 DataFrame에 넣음

      df = pd.DataFrame(rs)
      print(df)

finally:

   conn.close()

     total_bill   tip     sex smoker   day    time  size
0         16.99  1.01  Female     No   Sun  Dinner     2
1         10.34  1.66    Male     No   Sun  Dinner     3
2         21.01  3.50    Male     No   Sun  Dinner     3
3         23.68  3.31    Male     No   Sun  Dinner     2
4         24.59  3.61  Female     No   Sun  Dinner     4
..          ...   ...     ...    ...   ...     ...   ...
243       18.78  3.00  Female     No  Thur  Dinner     2
244       16.99  1.01  Female     No   Sun  Dinner     2
245       26.88  3.12    Male     No        Dinner     4
246       26.88  3.12    Male     No        Dinner     4
247       26.88  3.12    Male     No        Dinner     5

[248 rows x 7 columns]


In [5]:
df['tip_rate']=df.tip/df.total_bill

In [7]:
df.head()

,total_bill,tip,sex,smoker,day,time,size,tip_rate
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


In [8]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
tip_rate      0
dtype: int64

In [9]:
df.replace('',np.nan,inplace=True) 

In [10]:
df.sex.replace({'Female':0,'Male':1},inplace=True)

In [11]:
df.smoker.replace({'No':0,'Yes':1},inplace=True)
df.day.replace({'Thur':0,'Fri':1,'Sat':2,'Sun':3},inplace=True)
df.time.replace({'Lunch':0,'Dinner':1},inplace=True)

In [12]:
df.head()

,total_bill,tip,sex,smoker,day,time,size,tip_rate
0,16.99,1.01,0,0,3.0,1,2,0.059447
1,10.34,1.66,1,0,3.0,1,3,0.160542
2,21.01,3.50,1,0,3.0,1,3,0.166587
3,23.68,3.31,1,0,3.0,1,2,0.139780
4,24.59,3.61,0,0,3.0,1,4,0.146808


In [13]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           3
time          0
size          0
tip_rate      0
dtype: int64

In [ ]:
type(df)

In [14]:
# from numpy import isnan
from sklearn.impute import SimpleImputer

In [15]:
data = df.values

In [16]:
df.describe()

,total_bill,tip,sex,smoker,day,time,size,tip_rate
count,248.000000,248.000000,248.000000,248.000000,245.000000,248.000000,248.000000,248.000000
mean,19.860484,2.991734,0.645161,0.375000,1.730612,0.725806,2.588710,0.159853
std,8.866048,1.378257,0.479432,0.485102,1.156274,0.447009,0.965075,0.061110
min,3.070000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.035638
25%,13.385000,2.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.125340
50%,17.815000,2.960000,1.000000,0.000000,2.000000,1.000000,2.000000,0.153846
75%,24.527500,3.520000,1.000000,1.000000,3.000000,1.000000,3.000000,0.190658
max,50.810000,10.000000,1.000000,1.000000,3.000000,1.000000,6.000000,0.710345


In [17]:
# flatten() 여러 칼럼을 하나로 합친다. 
sum(np.isnan(data).flatten())

3

In [18]:
imputer = SimpleImputer(strategy='median')

In [19]:
# fit on the dataset
imputer.fit(data)

SimpleImputer(strategy='median')

In [20]:
# transform the dataset
data_trans = imputer.transform(data)

In [21]:
df_trans=pd.DataFrame(data_trans)

In [22]:
df_trans.isnull().sum()

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
dtype: int64

## RFE 적용

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  248 non-null    float64
 1   tip         248 non-null    float64
 2   sex         248 non-null    int64  
 3   smoker      248 non-null    int64  
 4   day         245 non-null    float64
 5   time        248 non-null    int64  
 6   size        248 non-null    int64  
 7   tip_rate    248 non-null    float64
dtypes: float64(4), int64(4)
memory usage: 15.6 KB


In [35]:
y=df.tip

In [25]:
X=df.drop('tip',axis=1)

In [26]:
X

,total_bill,sex,smoker,day,time,size,tip_rate
0,16.99,0,0,3.0,1,2,0.059447
1,10.34,1,0,3.0,1,3,0.160542
2,21.01,1,0,3.0,1,3,0.166587
3,23.68,1,0,3.0,1,2,0.139780
4,24.59,0,0,3.0,1,4,0.146808
...,...,...,...,...,...,...,...
243,18.78,0,0,0.0,1,2,0.159744
244,16.99,0,0,3.0,1,2,0.059447
245,26.88,1,0,NaN,1,4,0.116071
246,26.88,1,0,NaN,1,4,0.116071


In [27]:
y.describe()

count    248.000000
mean       2.991734
std        1.378257
min        1.000000
25%        2.000000
50%        2.960000
75%        3.520000
max       10.000000
Name: tip, dtype: float64

In [32]:
data=X.values

In [36]:
y=y.values

In [37]:
sum(np.isnan(data).flatten())

3

In [38]:
sum(np.isnan(y).flatten())

0

In [39]:
imputer = SimpleImputer(strategy='median')

In [40]:
imputer.fit(data)

SimpleImputer(strategy='median')

In [41]:
data_trans=imputer.transform(data)

In [42]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVR

In [43]:
estimator=SVR(kernel='linear')

In [44]:
rfe = RFE(estimator, n_features_to_select=4)

In [45]:
data_trans.shape

(248, 7)

In [46]:
rfe.fit(data_trans,y)

RFE(estimator=SVR(kernel='linear'), n_features_to_select=4)

In [70]:
for i in range(X.shape[1]):
  print('Column: %10s, Rank: %d' % (X.columns[i], rfe.ranking_[i]))

Column: total_bill, Rank: 1
Column:        sex, Rank: 1
Column:     smoker, Rank: 1
Column:        day, Rank: 3
Column:       time, Rank: 2
Column:       size, Rank: 4
Column:   tip_rate, Rank: 1


In [48]:
X.columns

Index(['total_bill', 'sex', 'smoker', 'day', 'time', 'size', 'tip_rate'], dtype='object')

## MinMaxScaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
trans=MinMaxScaler()

In [ ]:
X_norm=trans.fit_transform(data)

In [ ]:
df_norm=pd.DataFrame(X_norm)

In [ ]:
df_norm.describe()

## StandardScaler

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc=StandardScaler()

In [ ]:
df_sc = sc.fit_transform(data)

In [ ]:
df_transform_sc=pd.DataFrame(df_sc)

In [ ]:
df_transform_sc.describe().round()

In [ ]:
data_trans

## PCA

In [49]:
from sklearn.decomposition import PCA

In [50]:
trans = PCA(n_components=4)
data_dim = trans.fit_transform(data_trans)

In [51]:
trans.fit(data_dim)

PCA(n_components=4)

In [52]:
trans.explained_variance_ratio_

array([0.97205058, 0.01781401, 0.00750253, 0.00263288])

In [53]:
print(data_dim[:3,:])

[[-2.87701703 -1.26839191 -0.39790697  0.31216993]
 [-9.43648663 -1.63548501  0.96506813 -0.27783332]
 [ 1.20662897 -1.31436482  0.31716947 -0.35679409]]


In [75]:
trans.components_.round()

array([[ 1.,  0.,  0.,  0.],
       [-0.,  1., -0.,  0.],
       [-0.,  0.,  1., -0.],
       [ 0., -0., -0.,  1.]])

In [78]:
pca_x=pd.DataFrame(data_dim)

In [80]:
pca_x.describe().round(2)

,0,1,2,3
count,248.00,248.00,248.00,248.00
mean,0.00,0.00,-0.00,-0.00
std,8.89,1.20,0.78,0.46
min,-16.85,-1.64,-2.07,-0.77
25%,-6.51,-1.10,-0.44,-0.42
50%,-2.02,-0.38,-0.14,0.02
75%,4.72,1.19,0.40,0.27
max,30.91,2.54,3.11,1.05


In [82]:
columns = ['pca_comp_%i' % i for i in range(4)]
df_pca  = pd.DataFrame(trans.fit(data_dim).transform(pca_x), columns=columns, index=pca_x.index)
df_pca.head()

,pca_comp_0,pca_comp_1,pca_comp_2,pca_comp_3
0,-2.877017,-1.268392,-0.397907,0.312170
1,-9.436487,-1.635485,0.965068,-0.277833
2,1.206629,-1.314365,0.317169,-0.356794
3,3.804221,-1.161190,-0.812600,-0.518934
4,4.835133,-1.185388,1.075999,0.540980


## 회귀분석

In [54]:
# pearson's correlation feature selection for numeric input and numeric output
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [55]:
# define feature selection
fs = SelectKBest(score_func=f_regression, k=4)

In [56]:
# apply feature selection
X_selected = fs.fit_transform(data_trans, y)

In [72]:
fs.get_support(indices=True)

array([0, 3, 5, 6], dtype=int64)

In [57]:
print(X_selected.shape)

(248, 4)


In [58]:
pd.DataFrame(X_selected)

,0,1,2,3
0,16.99,3.0,2.0,0.059447
1,10.34,3.0,3.0,0.160542
2,21.01,3.0,3.0,0.166587
3,23.68,3.0,2.0,0.139780
4,24.59,3.0,4.0,0.146808
...,...,...,...,...
243,18.78,0.0,2.0,0.159744
244,16.99,3.0,2.0,0.059447
245,26.88,2.0,4.0,0.116071
246,26.88,2.0,4.0,0.116071
